In [1]:
import redivis
import numpy as np
import geopandas as gpd

# Prepare resistivity profiles for Step 1

- Download corresponding resistivity profiles from the online database: https://doi.org/10.57761/v480-xw80 
- Export to a format that can be readable to Leapfrog
 

## Download resistivity profiles

In [2]:
org_em_recharge = redivis.organization("KnightAccelerator")
em_dataset = org_em_recharge.dataset("em_data")
sediment_type_dataset = org_em_recharge.dataset("sediment_type_data")
waterlevel_dataset = org_em_recharge.dataset("waterlevel_data")
ca_geo_dataset = org_em_recharge.dataset("California State Geo Features")
target_path = "./"

In [5]:
import redivis

In [6]:
redivis.__version__

'0.11.1'

In [3]:
gdf_cv = gpd.read_file("./data/shp/cv.shp")
geom = gdf_cv.geometry[0]

from shapely.geometry import Polygon

USER_DEFINED_POLYGON = gdf_cv.geometry[0]
xx, yy = USER_DEFINED_POLYGON.exterior.coords.xy

# Create polygon from lists of points
x = xx[::1].tolist()
y = yy[::1].tolist()

USER_DEFINED_POLYGON = Polygon(zip(x,y))

In [4]:
# Then draw a polygon
SURVEY_ID = "cdwr_sacv"

q = em_dataset.query(f"""
    SELECT RECORD, ELEVATION, UTMX, UTMY, LINE_NO, SURVEY_ID, DOI_STANDARD, RESDATA, MEASUREMENTS
    FROM SurveyLocation L
    WHERE SURVEY_ID="{SURVEY_ID}" AND ST_Contains(ST_GeogFromText("{USER_DEFINED_POLYGON}"),L.geometry)
""")
df_em = q.to_dataframe()
df_em.to_csv(f"{target_path}/em_resistivity.csv", index=False)

print(f"Completed: {len(df_em.index)} rows downloaded.")

  0%|          | 0/376760 [00:00<?, ?it/s]

Completed: 376760 rows downloaded.


In [9]:
print("Loading... Layer Thickness to the compute server")
# TODO: add a season in the Survey table
q = em_dataset.query(f"""
    SELECT THICKNESS, KIND, YEAR, SEASON
    FROM Survey
    WHERE ID="{SURVEY_ID}" AND GEOTYPE="BOUNDS"
""")
df_thickness = q.to_dataframe()
df_thickness.to_csv(f"{target_path}/thickness.csv", index=False)

print(f"Completed.")

Loading... Layer Thickness to the compute server


  0%|          | 0/1 [00:00<?, ?it/s]

Completed.


## Export to files readable in Leapfrog

In [11]:
from emrecharge.datasets import EMDataset

In [12]:
SPATIAL_UNIT = 'm'
em_data = EMDataset(
    "./em_resistivity.csv", 
    "./thickness.csv",
    SPATIAL_UNIT
)

In [18]:
em_data.timestamps

array([     1,      2,      2, ..., 132016, 132017, 132018])

In [15]:
from SimPEG.electromagnetics.utils.em1d_utils import Stitched1DModel

In [21]:
Stitched1DModel?

Init signature:
Stitched1DModel(
    topography=None,
    physical_property=None,
    line=None,
    time_stamp=None,
    thicknesses=None,
    **kwargs,
)
Docstring:      <no docstring>
File:           ~/Project/simpeg/SimPEG/electromagnetics/utils/em1d_utils.py
Type:           type
Subclasses:     

In [28]:
_, line_number = np.unique(em_data.line, return_inverse=True)

In [30]:
line_number.shape

(376760,)

In [32]:
stitched_model = Stitched1DModel(
    thicknesses=em_data.hz[:-1],
    line=line_number,
    topography=em_data.topography,
    time_stamp=em_data.timestamps,
)

In [49]:
from SimPEG import utils
import pandas as pd
def write_aem_leapfrog_csv(
    hz, Line, topography, time_stamp, properties,
    name_properties=['resistivity'],
    nskip=1, work_dir='./', fname_header=''
):

    n_sounding_all = topography.shape[0]
    stitched_model = Stitched1DModel(
        thicknesses=hz[:-1],
        line=Line[::nskip],
        topography=topography[::nskip,:].copy(),
        time_stamp=time_stamp[::nskip]
    )
    depth_from = np.zeros((hz.size, stitched_model.n_sounding), order='F')
    depth_to = np.zeros((hz.size, stitched_model.n_sounding), order='F')

    # collar information
    lineid_collar = Line[::nskip]
    wellid_collar = np.arange(lineid_collar.size) + 1

    north_collar = stitched_model.topography[:,1]
    east_collar = stitched_model.topography[:,0]
    depth_max_collar = np.ones_like(wellid_collar) * hz.sum()
    elevation_collar = stitched_model.topography[:,2]

    # interval information

    for ii in range(stitched_model.n_sounding):
        depth_from[:,ii] = stitched_model.mesh_1d.vectorNx[:-1]
        depth_to[:,ii] = stitched_model.mesh_1d.vectorNx[1:]

    lineid = stitched_model.line.repeat(hz.size)
    wellid = wellid_collar.repeat(hz.size)
    east = utils.mkvc(stitched_model.xyz[:,0])
    north = utils.mkvc(stitched_model.xyz[:,1])
    depth_from = utils.mkvc(depth_from[:,:])
    depth_to = utils.mkvc(depth_to[:,:])

    df_collar = pd.DataFrame(data=np.c_[wellid_collar, stitched_model.line, east_collar, north_collar, elevation_collar, depth_max_collar], columns=['wellid', 'Line', 'X', 'Y', 'elevation', 'depth'])
    df_collar.to_csv(work_dir+'collar-{:s}.csv'.format(fname_header), index=False)


    distance_collar = np.zeros_like(wellid_collar)
    azimuth_collar = np.zeros_like(wellid_collar)
    dip_collar = np.ones_like(wellid_collar) * 90.

    df_survey = pd.DataFrame(data=np.c_[wellid_collar, distance_collar, azimuth_collar, dip_collar], columns=['wellid', 'distance', 'azimuth', 'dip'])
    df_survey.to_csv(work_dir+'survey-{:s}.csv'.format(fname_header), index=False)
    data = np.c_[wellid, depth_from, depth_to]
    for ii in range (len(name_properties)):
        data = np.c_[data, properties[:,ii].reshape((n_sounding_all, hz.size))[::nskip, :].flatten()]
    df_interval = pd.DataFrame(
        data=data,
        columns=['wellid', 'from', 'to'] + name_properties
    )
    df_interval.to_csv(work_dir+'interval-{:s}.csv'.format(fname_header), index=False)
    return df_collar, df_survey, df_interval

In [50]:
rho = em_data.resistivity.flatten()

In [51]:
properties = rho.reshape([-1,1])

In [54]:
write_aem_leapfrog_csv(
    em_data.hz, line_number, em_data.topography, em_data.timestamps, properties,
    nskip = 10, fname_header='cdwr', work_dir='./outputs/leapfrog_files/'
)

/Users/sgkang09/anaconda3/envs/em_recharge/lib/python3.8/site-packages/discretize/utils/code_utils.py:216: FutureWarning: TensorMesh.vectorNx has been deprecated, please use TensorMesh.nodes_x. It will be removed in version 1.0.0 of discretize.
  warnings.warn(message, warn)
/Users/sgkang09/anaconda3/envs/em_recharge/lib/python3.8/site-packages/discretize/utils/code_utils.py:216: FutureWarning: TensorMesh.vectorCCx has been deprecated, please use TensorMesh.cell_centers_x. It will be removed in version 1.0.0 of discretize.
  warnings.warn(message, warn)


(        wellid   Line              X              Y  elevation    depth
 0          1.0  300.0 -181331.375000   75145.796875      135.4  674.451
 1          2.0  300.0 -181339.140625   75301.929688      156.3  674.451
 2          3.0  300.0 -181303.531250   75457.859375      171.1  674.451
 3          4.0  300.0 -181264.281250   75609.164062      160.3  674.451
 4          5.0  300.0 -181236.234375   75982.367188      114.6  674.451
 ...        ...    ...            ...            ...        ...      ...
 37671  37672.0  299.0   72250.554688 -168114.484375      123.4  674.451
 37672  37673.0  299.0   72510.398438 -167896.921875      125.2  674.451
 37673  37674.0  299.0   72797.828125 -167722.281250      129.3  674.451
 37674  37675.0  299.0   73114.640625 -167721.031250      131.9  674.451
 37675  37676.0  299.0   73417.992188 -167666.625000      135.3  674.451
 
 [37676 rows x 6 columns],
         wellid  distance  azimuth   dip
 0          1.0       0.0      0.0  90.0
 1          2